In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lecture 8 - Clustering with k-Means  *
---


### Content

1. What is unsupervised learning?
2. Clustering with k-Means
3. Evaluating clustering - cluster cohesion and separation
4. Using scikit-learn for clustering


### Learning Outcomes

At the end of this lecture, you should be able to:

* explain the motivation behind clustering
* recognise the types of problems that unsupervised machine learning (clustering) is applied to 
* explain the k-means clustering algorithm
* apply the k-means algorithm and interpret its output
* evaluate the results of clustering
* apply clustering from scikit-learn


---

\* Some material on clustering is sourced from Janert, P. K. (2010). Data analysis with open source tools. O'Reilly Media, Inc

---

# Unsupervised Machine Learning (Clustering)

So far we have learned how to use **supervised machine learning** techniques like classification and regression. 

In supervised learning, we new exactly what the inputs and its expected outputs were. We would provide our machine learning algorithm with the **inputs** and ask it to **learn** to uncover the underlying patterns which would allow it to **correctly map to the provided corresponding outputs**. 

When deploying a classifier of this type, we would subsequently provide it with inputs that it has not seen before and expect it to **produce informed outputs** (class labels or continuous values) based on what the classifier has learned before.

**Clustering** is a family of **unsupervised machine learning techniques**. It operates in a fundamentally different way to classification and regression. 

Clustering is a **method for discovering and visualising distinct groups** of similar data points. The purpose of clustering is to **find structure** and **discover patterns** in a dataset where no one piece of data within it provides a definitive answer. 

Clustering is often seen as an **exploratory method** which is a computationally-driven approach to discovering structure in data. It is to a large degree subjective and requires a strong understanding of the problem domain in order to ascribe meaning to the results.  The result of clustering are always sets of clusters; however, this does not necessarily mean that the clusters are significant, possess any real meaning and thus are present in the real-world domain from which the data originated. Many times the results will be devoid of meaning or very hard to make sense of. 

Clustering is heavily used in data-intensive domains. Examples of usage: groups of customers with similar buying patterns can automatically be detected by retailers who track customer purchases (loyalty/reward cards) and **marketing or retail strategies** can be developed from this; identification of areas of similar land use in an **earth observation** database; in **insurance** settings, groups of motor insurance policy holders with a high average claim cost can be identified; groups of similar houses according to their house type, value, and geographical location can be identified in **city planning** scenarios; in **medicine**, clustering can be used in automatically identifying cancerous cells in datasets without any prior labelling; **computational biology** also employs clustering to discover groups of genes that exhibit similar behaviour, which might indicate that they respond to a treatment in the same way or are part of the same biological pathway.


## What is a cluster?

The concept of a "cluster" is in fact not well defined and lacks theoretical rigour.

Simply saying **"a cluster is a set of similar points"** is in many ways **insufficient**. The same goes for **"a cluster is a group of points that are close together"**. The reason that these descriptions are insufficient is because clusters **must also be well separated** from each other.

![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_uniform.jpg)

Source: Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc


An arguably more precise definition of what a cluster is: **"contiguous regions of high data point density separated by regions of lower point density".**


![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_smile.jpg)

> Source: Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc

The previous definition of what regions of clusters should be opens up the possibilities for many types of very different clusters.

The image below depicts regions of clusters made up of **graph-like relationships** rather than **point density**. In addition, this example depicts  scenarios where we observe **nested clusters** within other clusters. 

![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_graph.jpg)

> Source: Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc


Cluster analysis does not need to be limited to just points in space, but can involve **strings as well non-geometric data like time-series**. The challenge in these scenarious then becomes of **how we define the notion of "similarity".** 

![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_strings_time_series.jpg)

> Source: Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc

Final examples of clusters which are easily discernible by human eyes but difficult to train computers to do are expressed in the images below. The **intertwining of the clusters** in one dataset and the **intersection of two clusters** in the other are a particular **challenge** to clustering algorithms and not trivially solved.


![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_complexity.jpg)

> Source: Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc

### Example of Cluster Formation with Different Algorithms

A good article and illustration of five key clustering algorithms, with their approaches to forming clusters can be found here: https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68




![the-5-clustering-algorithms-data-scientists-need-to-know](https://miro.medium.com/max/960/1*KrcZK0xYgTa4qFrVr0fO2w.gif)

![the-5-clustering-algorithms-data-scientists-need-to-know](https://miro.medium.com/max/552/1*bkFlVrrm4HACGfUzeBnErw.gif)

![the-5-clustering-algorithms-data-scientists-need-to-know](https://miro.medium.com/max/864/1*vyz94J_76dsVToaa4VG1Zg.gif)

![the-5-clustering-algorithms-data-scientists-need-to-know](https://miro.medium.com/max/1104/1*tc8UF-h0nQqUfLC8-0uInQ.gif)

![the-5-clustering-algorithms-data-scientists-need-to-know](https://miro.medium.com/max/720/1*OyXgise21a23D5JCss8Tlg.gif)

![the-5-clustering-algorithms-data-scientists-need-to-know](https://miro.medium.com/max/1280/1*ET8kCcPpr893vNZFs8j4xg.gif)

> Source:George Seif https://towardsdatascience.com/the-5-clustering-algorithms-data-scientists-need-to-know-a36d136ef68

## Distance and Similarity Measures

In the same way that we defined distance metrics in our usage of the kNN algorithm, the **clustering algorithms also require the definition of a function that returns a scalar value to denote distance or similarity between two points**. Whether the function expresses the distance or similarity between two points is a matter of choice and the domain - distance can be transformed into similarity and vice versa.

The image below shows examples of some of the popular distance and similarity measures, including some we have already covered in this course. Despite its simplicity, the **Euclidean** distance metric remains one of the most widely used on numerical domains.

Just as we discovered that data comprising of **different scales** can have a **negative effect** on the kNN algorithm, the same is true in clustering. Whenever we employ distance metrics like Euclidean, we **must perform normalization** before applying the clustering algorithm. Correlation-based similarity metrics are however resistant to range variability within data.


![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_distance_similarity.jpg)

> Source: Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc


## Clustering Algorithms

There are different families of clustering algorithms. 

### Tree Builders

Tree building clustering algorithms construct **decision tree-like structures** by successively combining clusters that are “close” to each
other into a larger cluster until only a single cluster remains. This technique is known as
**agglomerative hierarchical clustering**. The final cluster representation is a tree-like hierarchy of clusters. 

Clusters that exhibit similarity are merged early, constituting the leaves part of a tree. More dissimilar 
clusters are joined later in the process, nearer the root of the tree. 

The resulting structure can be represented graphically in a **dendrogram** (seen in the image below that depicts clusters of blogs based on word usage from which topics emerge). To extract actual clusters from it, we need to walk the tree, evaluate the cluster properties for each subtree,
and then cut the tree to obtain clusters.

![Source Janert, P. K. (2010). Data analysis with open source tools.  O'Reilly Media, Inc](../figures/cluster_dendrogram.jpg)

Tree building clustering algorithms are very **computationally intensive**. However, their advantage is that they do not just produce a flat list of clusters, but instead **explicitly show the relationships** between the clusters that can then be interpreted for meaning.



### Centre Seekers

The most widely used category of clustering algorithms is the **k-means** family. 

The k-means algorithm is an **iterative algorithm**. The algorithm requires at the outset that the number of **expected clusters k be specified** as input to it. The key principle behind the algorithm is to **repeatedly re-calculate the centre or the centroid of each cluster**, and re-assign all the points to each centroid depending on their distance from it. This process is repeated until a predefined convergence is achieved or the maximum number of iterations are carried out.


![Segaran, T. (2007). Programming collective intelligence: building smart web 2.0 applications.  O'Reilly Media, Inc.](../figures/cluster_kmeans.jpg)


> Image Source: Segaran, T. (2007). Programming collective intelligence: building smart web 2.0 applications. " O'Reilly Media, Inc.".

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mtpl
import math 
import matplotlib.pyplot as plt
import random
from sklearn import preprocessing

%matplotlib inline

df = pd.io.parsers.read_csv(
    '../datasets/wine_data.csv',
     usecols=[0,6,7]
    )

df.columns=['Class','Magnesium','Flavanoids']
df.head()

In [ ]:
fig, axes = plt.subplots()
fig.set_size_inches(10,10)

axes.grid()
axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)
axes.scatter(df.Magnesium, df.Flavanoids, s=90, alpha=0.6, c='red')

We first need to normalise the data:

**Exercise:** Re-scale the 'Magnesium' and 'Flavanoids' values using MinMax.

The cetroid is critical to understanding k-means. It is the average position of all the points in a set of data instances. In our often n-dimensional spaces, the centroid is the mean position of all the points in all of the coordinate directions i.e. it is the mean of each feature type.

**Exercise 1:** Find the centroid for all the data points in the above graph and plot it as a black dot in the graph:

In [ ]:
fig, axes = plt.subplots()
fig.set_size_inches(10,10)
axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)

# YOUR CODE HERE


The basic algorithm can be summarized as follows: 

1. Randomly pick **k** centroids (or points that will be the center of your clusters) in multi dimensional d-space. Strive to make them near the data but away from each another.
2. Assign each data point to the closest centroid.
3. Move the centroids to the average location of the data points assigned to it.
4. Repeat the preceding two steps until the assignments no longer change - or change very little.

When randomly selecting the initial centroid(s) for the kmeans algorithm, it is important that the randomly chosen centroid is within the data range for each of the feature types.

**Exercise 2:**  Using the data above, randomly select a centroid that is within the range of values for Flavanoids and Magnesium and plot it as a black dot. (Use random.uniform(x_min,x_max) to generate a random number in the range x_min amd x_max)

In [ ]:
fig, axes = plt.subplots()
fig.set_size_inches(10,10)

axes.grid()
axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)
axes.scatter(df.Magnesium, df.Flavanoids, s=90, alpha=0.6, c='red')

# YOUR CODE HERE


**Exercise 3:** Given two centroids ([0.1,0.6] and [0.3,0.6]) in the graph below, calculate the closest points to each of the centroids using the Euclidean distance defined below. Then, calculate the centroid of each of the samples assigned to each of the centroids. Make the centroids of each of the two groups of points the new centroid, and plot

In [ ]:
def eucledean_distance(x, y):
    d = 0.0
    
    for i in range(len(x)):
        d += (x[i] - y[i])**2
    d = math.sqrt(d)
    
    return d

In [ ]:
fig, axes = plt.subplots()
fig.set_size_inches(10,10)
axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)
axes.scatter(df.Magnesium, df.Flavanoids, s=90, alpha=0.6, c='red')
axes.scatter(0.1, 0.6, s=150, alpha=1, c='black')
axes.scatter(0.3, 0.6, s=150, alpha=1, c='black')

In [ ]:
fig, axes = plt.subplots()
fig.set_size_inches(10,10)

axes.grid()
axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)
axes.scatter(df.Magnesium, df.Flavanoids, s=90, alpha=0.6, c='red')

counts = [0,0]                          # keep track of how many points are closer to centroid 0 or 1
centroids = [[0.1, 0.6], [0.3, 0.6]]    # starting points for centroids
centroid_coordinates = [[0,0],[0,0]]    # update these centroid coordinates based on closest points

rows = df[['Magnesium','Flavanoids']].values

for j in range(len(rows)):
    row = rows[j]         # current data point
    best_match = 0        # which centroid is the closest? default = 0
    for i in range(2):    # for each data point calculate if it is closer to centroid 0 or 1
       # YOUR CODE HERE


centroid_coordinates[0] /=  counts[0]
centroid_coordinates[1] /=  counts[1]


axes.scatter(centroid_coordinates[0][0], centroid_coordinates[0][1], s=150, alpha=1, c='black')
axes.scatter(centroid_coordinates[1][0], centroid_coordinates[1][1], s=150, alpha=1, c='black')
print(centroids)
print(centroid_coordinates)

The k-means algorithm is **nondeterministic**. This means that different starting values may produce very **different results**. Given this, it is expected that the algorithm be **run multiple times** with different randomly selected starting values and to then compare results.

**k-means cannot be used for categorical data** unless the distance/similarity function is redefined for categorical data. One option is to use Jaccard similarity:

In [ ]:
def jaccard_similarity(x,y):
    x = set(x)
    y = set(y)
    intersection = len(set.intersection(x,y))
    union = len(set.union(x, y))
    
    return intersection / float(union)

In [ ]:
1 - jaccard_similarity(['apples','bananas','oranges','kiwi','chrries'],['apples','bananas','oranges','grapefruit','grapes'])

In [ ]:
1 - jaccard_similarity(['apples','bananas','oranges','kiwi','chrries'],['apples','berries','pears','grapefruit','grapes'])

In [ ]:
def categorical_variable_distance(s1, s2):
    if len(s1) != len(s2):
        raise ValueError("Sequences are of unequal length")
    number_of_matches = sum(ch1 == ch2 for ch1, ch2 in zip(s1, s2))
    
    return (number_of_matches) / float(len(s1))

In [ ]:
1 - categorical_variable_distance(['apples','bananas','oranges','kiwi','chrries'],['apples','bananas','oranges','grapefruit','grapes'])

In [ ]:
1 - categorical_variable_distance(['apples','bananas','oranges','kiwi','chrries'],['apples','berries','pears','grapefruit','grapes'])

### Issues with k-means:

- **Choosing k is more an art than a science**, although there are bounds: 1≤k ≤n, where n is number of data points.

- There are **convergence issues** — the solution can fail to exist, if the algorithm falls into a loop, for example, and keeps going back and forth between two possible solutions, or in other words, there isn’t a single unique solution.

- **Interpretability** can be a problem—sometimes the answer isn’t at all useful. Indeed that’s often the biggest problem. In spite of these issues, it’s pretty fast (compared to other clustering algorithms), and there are broad applications in marketing, computer vision (partitioning an image), or as a starting point for other models

> Schutt, R., & O'Neil, C. (2013). Doing Data Science: Straight Talk from the Frontline. " O'Reilly Media, Inc.".

Below is adapted python code for kmeans from:

> Segaran, T. (2007). Programming collective intelligence: building smart web 2.0 applications. " O'Reilly Media, Inc.".

In [ ]:
fig, axes = plt.subplots()
fig.set_size_inches(10,10)

axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)
axes.grid()
axes.scatter(df.Magnesium.where(df.Class == 1), df.Flavanoids.where(df.Class == 1), s=90, alpha=0.6, c='red')
axes.scatter(df.Magnesium.where(df.Class == 2), df.Flavanoids.where(df.Class == 2), s=90, alpha=0.6, c='blue')
axes.scatter(df.Magnesium.where(df.Class == 3), df.Flavanoids.where(df.Class == 3), s=90, alpha=0.6, c='green')


axes.scatter(0,0.9, s=150, alpha=1, c='black')
axes.scatter(0.1,0.6, s=150, alpha=1, c='black')
axes.scatter(0, 0.3, s=150, alpha=1, c='black')

In [ ]:
def kmeans_cluster(rows, distance = eucledean_distance, k = 3, iter = 10):
    
    # Determine the minimum and maximum values for each point
    ranges=[(min([row[i] for row in rows]),max([row[i] for row in rows])) for i in range(len(rows[0]))]
    print(ranges)
    
    # Create k randomly placed centroids
    #centroids=[[random.random( ) * (ranges[i][1] - ranges[i][0]) + ranges[i][0]  for i in range(len(rows[0]))] for j in range(k)]
    centroids=[[0,0.9],[0.1,0.6],[0, 0.3]]
    print(centroids)
    
    prev_cluster_labels = None
    for t in range(iter):
        #print('Iteration %d' % t)
        cluster_labels = [[] for i in range(k)]
        #print(cluster_labels)
        # Find which centroid is the closest for each row
        for j in range(len(rows)):
            row = rows[j]
            best_match = 0
            for i in range(k):
                d = distance(centroids[i],row)
                if d < distance(centroids[best_match],row): 
                    best_match=i
            cluster_labels[best_match].append(j)
            
        # If the results are the same as last time, this is complete
        if cluster_labels == prev_cluster_labels:
            break
        prev_cluster_labels = cluster_labels
    
        # Move the centroids to the average of their members
        for i in range(k):
            avgs = [0.0] * len(rows[0])
            if len(cluster_labels[i]) > 0:
                for rowid in cluster_labels[i]:
                    for m in range(len(rows[rowid])):
                        avgs[m] += rows[rowid][m]
                for j in range(len(avgs)):
                    avgs[j] /= len(cluster_labels[i])
                centroids[i] = avgs
            
    return cluster_labels, centroids

In [ ]:
labels, centroids = kmeans_cluster(rows=df[['Magnesium','Flavanoids']].values,k = 3, iter = 3)
#print labels, centroids
fig, axes = plt.subplots()
fig.set_size_inches(10,10)

axes.set_xlabel('Magnesium',fontsize=30)
axes.set_ylabel('Flavanoids',fontsize=30)
plt.title('Wine Dataset',fontsize=30)
axes.grid()

axes.scatter(df.Magnesium.where(df.Class == 1), df.Flavanoids.where(df.Class == 1), s=90, alpha=0.6, c='red')
axes.scatter(df.Magnesium.where(df.Class == 2), df.Flavanoids.where(df.Class == 2), s=90, alpha=0.6, c='blue')
axes.scatter(df.Magnesium.where(df.Class == 3), df.Flavanoids.where(df.Class == 3), s=90, alpha=0.6, c='green')


axes.scatter(centroids[0][0],centroids[0][1], s=150, alpha=1, c='black')
axes.scatter(centroids[1][0], centroids[1][1], s=150, alpha=1, c='black')
axes.scatter(centroids[2][0], centroids[2][1], s=150, alpha=1, c='black')

plt.show()

**Exercise:** Evaluate the classification accuracy of the kmeans clustering above.

### Evaluating clustering

#### Cohesion and Separation

We can evaluate each cluster based and its **cohesion** value. Cohesion describes the **closeness of the points within a cluster**. It is **average distance** of all the points in the cluster to the cluster centroid.
 

**Separation** defines the average distance between a cluster and all the points outside of the cluster.

Given the definition of the cohesion and separation for a single cluster, this can easily be extended to produce a measure of the overall clustering for a given problem. We can use the average cohesion and average separation (preferably weighted by each cluster mass which is the number of samples in each cluster) to give us global metrics for evaluating a clustering solution.  
 
![.](../figures/cluster_cohesion_separation.jpg)


Once we have calculated the global cohesion and separation values for each cluster, we can combine them into a single value that represents the ratio:

$\frac{separation}{cohesion}$

where we expect the ratio to increase as the separability and the cohesion of a clustering solution improve.

> Janert, P. K. (2010). Data analysis with open source tools. O'Reilly Media, Inc


**Exercise 4:** The kmeans_cluster cluster function above returned a list of labels (indexes) of all samples assigned to each cluster as well as a list of centroids for each cluster. Calculate the cohesion for cluster [0].

In [ ]:
labels[0]


In [ ]:
centroids

In [ ]:
rows = df[['Magnesium','Flavanoids']].values

dist = 0.0
mass = 0

#YOUR CODE HERE
for row in labels[0]:


dist / float(mass)

**Exercise 5:**  Calculate the separation of cluster [0] to elements in cluster [1].

In [ ]:
sep = 0.0
num = 0

#YOUR CODE HERE
for row in labels[1]:

    
sep / float(num)

**Exercise 6:**  Calculate the separation of cluster [0] to all clusters.

In [ ]:
total_sep = 0.0
num = 0

#YOUR CODE HERE
for c in range(1,3):


total_sep / float(num)

### Warning

- Clustering is fun and fascinating but it can also lead you astray and become a waste of time. Obtaining useful results is often difficult. Interpretation is a challenge.

- We tend to assume that our dataset has clusters, or a certain number, and this may prove to be wrong assumption.

- Assuming you have found clusters and are able to draw out some meaning from them, the field is still lacking rigorous tools to evaluate the findings and even to know exactly what to do with them.

- A good starting point with clustering is to formulate the question you are wanting to answer at the outset, together with hypotheses and then use the data to validate or disprove them.



### To summarize:

• The k-means algorithms and its variants work best for globular (at least star-convex)
clusters. The results will be meaningless for clusters with complicated shapes and for
nested clusters.

• The expected number of clusters is required as an input. If this number is not known, it
will be necessary to repeat the algorithm with different values and compare the results.

• The algorithm is iterative and nondeterministic; the specific outcome may depend on
the choice of starting values.

• The k-means algorithm requires vector data; use a different distance method for categorical data

• The algorithm can be misled if there are clusters of highly different size or different
density.

• The k-means algorithm is linear in the number of data points; the k-medoids algorithm
is quadratic in the number of points.

> Janert, P. K. (2010). Data analysis with open source tools. O'Reilly Media, Inc

## Clustering Using scikit-learn

In [ ]:
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [ ]:
km = KMeans(n_clusters=3, init='random')

In [ ]:
km.fit(df[['Magnesium','Flavanoids']].values)

In [ ]:
predictions = km.predict(df[['Magnesium','Flavanoids']].values)
predictions

Another useful measure that provides a useful global figure that expresses how well the clustering has performed is the **Silhouette Coefficient (SC)**.

The SC is calculated for each point as follows:

a = average distance to all other points in its cluster

b = average distance to all other points in the next nearest cluster

SC = (b-a)/max(a, b)

SC is in the range from -1 (worst) to 1 (best).

A global SC is calculated by taking the average of the SC for all points.

In [ ]:
from sklearn import metrics
metrics.silhouette_score(df[['Magnesium','Flavanoids']].values, predictions)

Since k-means essentially attempts to minimise the within-cluster sum of squares (WCSS), we can experimentally visualise this property for a range of k values in order to ascertain what is the ideal number of clusters - aiming for the smallest possible and the lowest WCSS.

In [ ]:
k_rng = range(1,10)
est = [KMeans(n_clusters = k).fit(df[['Magnesium','Flavanoids']].values) for k in k_rng]

# Generally want to minimize WSS, while also minimizing k
within_cluster_sum_squares = [e.inertia_ for e in est]
fig, axes = plt.subplots()
fig.set_size_inches(15,20)
# Plot the results
plt.subplot(212)
plt.plot(k_rng, within_cluster_sum_squares, 'b*-')
plt.xlim([1,10])
plt.grid(True)
plt.xlabel('k', fontsize=20)
plt.ylabel('Within Cluster Sum of Squares', fontsize=20)
plt.title('Within Cluster Sum of Squares versus number of Clusters', fontsize=20)

**Exercise:**

Given below is a dataset sourced from https://raw.githubusercontent.com/justmarkham/DAT7/master/data/beer.txt which contains information on properties of some of the most popular beer brands in the US:


In [ ]:
beer_df = pd.read_csv('../datasets/beer.csv')
beer_df


Your task it to perform clustering on this dataset and evaluate the meaning of the clusters. 